In [1]:
n = 5 * int(1e4)
pbs = 32
q = 0.5

alpha = 1e-9 # failure prob.

from scipy.stats import binom

k = 1
binom_dist = binom(n, q)
while True:
    right_prob = binom_dist.sf(k * pbs)
    if right_prob < alpha:
        break
    k += 1

In [2]:
binom_dist.sf(k * pbs)

2.3310141271004456e-10

In [3]:
binom_dist.sf((k-1) * pbs)

1.392372124750398e-09

In [4]:
k * pbs

25696

In [5]:
import jax
import jax.numpy as jnp
import numpy as np
from jax.lax import fori_loop
mlbs = k * pbs
Xs = jax.random.normal(jax.random.PRNGKey(123), (n,))

In [6]:
Xs

Array([-0.03668689, -1.1082411 ,  1.0482097 , ...,  0.31209594,
        0.12048835,  0.2659442 ], dtype=float32)

In [7]:
def single_iter(t_iter, args):
    choice_rng, binom_rng = jax.random.split(jax.random.PRNGKey(t_iter), 2)
    logical_batch = jax.random.choice(choice_rng, Xs, shape=(mlbs,), replace=False)
    physical_batches = jnp.array(jnp.split(logical_batch, k))
    
    actual_logical_bs = jax.random.bernoulli(binom_rng, q, shape=(n,)).sum()
    masks = jnp.array(jnp.split((jnp.arange(mlbs) < actual_logical_bs), k))

    def foo(t, args):
        cumulative_sum_so_far = args
        mask = masks[t]
        pb_sum = (physical_batches[t] * mask).sum()
        return cumulative_sum_so_far + pb_sum

    final_sum = fori_loop(0, k, foo, 0.)
    
    # add noise to grads
    # update parameters
    # instead of final sum, you would pass the updated params, or whatever to the next iteration
    return final_sum
final_result = fori_loop(0, 10, single_iter, 0.)

In [8]:
final_result

Array(85.314644, dtype=float32)

In [12]:
import torch
import torchvision
import torch.utils.data as data
import numpy as np

DATA_MEANS2 = (0.485, 0.456, 0.406)
DATA_STD2 =  (0.229, 0.224, 0.225)

def image_to_numpy_wo_t(img):
    img = np.array(img, dtype=np.float32)
    img = ((img / 255.) - DATA_MEANS2) / DATA_STD2
    img = np.transpose(img,[2,0,1])
    return img

transformation = torchvision.transforms.Compose([
        torchvision.transforms.Resize(224),
        image_to_numpy_wo_t,
        #torchvision.transforms.ToTensor(),
        #torchvision.transforms.Normalize(DATA_MEANS,DATA_STD),
    ])

train_set = torchvision.datasets.CIFAR10(root='../data_cifar10/',train=True,download=True,transform=transformation)
train_loader = data.DataLoader(train_set,batch_size=k*pbs,shuffle=True)

Files already downloaded and verified


In [13]:
for batch in train_loader:
    print('logical batch size',len(batch[1]))

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>